### 디버깅 또는 실행 속도 모드 설정
0 = debug(가장 느림), 1 = normal run, 2 = run fast, 3 = without camera view(가장 빠름)

In [1]:
runMode = 3

### OLED 디스플레이 (Adafruit OLED, SSD1306, 128 x 32)

In [2]:
import Adafruit_SSD1306
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

from jetbot.utils.utils import get_ip_address
import subprocess
import time

##### OLED-SSD1306 초기화

In [3]:
# 하드웨어 I2C 방식의 128x32 display
# gpio는 OLED 디바이스 인식문제 해결을 위해 1로 설정
disp = Adafruit_SSD1306.SSD1306_128_32(rst=None, i2c_bus=0, gpio=1)

# 디스플레이 초기화
disp.begin()

# 화면 클리어
disp.clear()
disp.display()

# 디스플레이 이미지 생성, 흑백(1-bit) 이미지 사용
disp_image = Image.new('1', (disp.width, disp.height))

# 이미지에 그릴 드로잉 객체 얻기
screen = ImageDraw.Draw(disp_image)

# 화면 전체를 검은색으로 채워서 지우기
screen.rectangle((0, 0, disp.width, disp.height), outline=0, fill=0)

# 화면 패딩영역 위치 조정(글씨가 짤리지 않도록)
screen_padding = 2
screen_top = screen_padding
screen_bottom = disp.height - screen_padding
screen_x = 0

# 기본 서체 사용
font = ImageFont.load_default()

##### OLED 화면에 기본 정보 디스플레이

In [4]:
### OLED 화면에 시스템 정보 4줄을 보여주는 함수

def ShowInfo(strInfo):
    # 화면 전체를 검은색으로 채워서 지우기
    screen.rectangle((0, 0, disp.width, disp.height), outline=0, fill=0)
    
    # 화면 맨 위의 첫번째 줄에는 매개변수로 넘겨받은 문자열을 디스플레이
    screen.text((screen_x, screen_top+0), strInfo, font=font, fill=255)
    
    # 두번째 줄에는 젯봇의 IP Address를 디스플레이
    screen.text((screen_x, screen_top+8), "IP:" + str(get_ip_address('wlan0')), font=font, fill=255)
    
    # 세번째 줄에는 메모리 사용 상태를 디스플레이
    cmd = "free -m | awk 'NR==2{printf \"Mem:%s/%sM %.2f%%\", $3, $2, $3*100/$2 }'"
    MemUsage = subprocess.check_output(cmd, shell = True)
    screen.text((screen_x, screen_top+16), str(MemUsage.decode('utf-8')), font=font, fill=255)
    
    # 네번째 줄에는 디스크 사용 상태를 디스플레이
    cmd = "df -h | awk '$NF==\"/\"{printf \"Disk:%d/%dGB %s\", $3, $2, $5}'"
    Disk = subprocess.check_output(cmd, shell = True)
    screen.text((screen_x, screen_top+25), str(Disk.decode('utf-8')), font=font, fill=255)
    
    # 설정한 내용을 OLED 화면으로 출력
    disp.image(disp_image)
    disp.display()
    
# 맨 위의 1번째 줄에 텍스트를 디스플레이 하는 함수
def ShowTopInfo(strInfo):
    screen.rectangle((0, 0, disp.width, 7), outline=0, fill=0)
    screen.text((screen_x, screen_top), strInfo, font=font, fill=255)
    disp.image(disp_image)
    disp.display()
    
if runMode < 2:
    ShowInfo("Init...")

In [5]:
ShowInfo('hello')

### RGB Strip LED 생성
젯봇 좌우의 무한궤도 캐터필러 안쪽에 장착되어 있는 5x2 LED 제어

In [6]:
from RGB_Lib import Programing_RGB
RGB = Programing_RGB()

#### 개별 1개 LED 또는 전체 LED 색상 설정

##### LED는 인덱스가 0번부터 9번까지 전부 10개
##### 오른쪽: 뒤쪽[0] > [1] > [2] > [3] > 앞쪽[4] / 왼쪽: 뒤쪽[5] > [6] > [7] > [8] > 앞쪽[9]
##### 뒷 페이지의 순차흐름, 무지개, 숨쉬기 LED Show는 생략

In [20]:
# if runMode < 2:
#     RGB.Set_An_RGB(0, 0xFF, 0x00, 0x00) # 0번째 LED (우측 뒤쪽) 빨강
#     RGB.Set_An_RGB(4, 0x00, 0xFF, 0x00) # 0번째 LED (우측 앞쪽) 초록
#     RGB.Set_An_RGB(5, 0x00, 0x00, 0xFF) # 0번째 LED (좌측 뒤쪽) 파랑
#     RGB.Set_An_RGB(9, 0xFF, 0xFF, 0xFF) # 0번째 LED (좌측 앞쪽) 흰색

#     RGB.Set_All_RGB(0xFF, 0x00, 0x00) # 전체 LED 색상을 설정
#     time.sleep(0.5)
#     RGB.Set_All_RGB(0x00, 0xFF, 0x00) # 전체 LED 색상을 설정
#     time.sleep(0.5)
#     RGB.Set_All_RGB(0x00, 0x00, 0xFF) # 전체 LED 색상을 설정
#     time.sleep(0.5)
    
# RGB.OFF_ALL_RGB() # 전체 LED 끄기
# time.sleep(1)


### I/O 제어: 부저, 버튼, 단색 LED

In [7]:
import RPi.GPIO as GPIO

BEEP_pin = 6 # Buzzer 핀 번호

GREEN_pin = 24 # BOARD pin 24, GREEN (LED2)
BLUE_pin = 23 # BOARD pin 23, Blue (LED3)

KEY1_pin = 8 # BOARD pin 8, K1 버튼
KEY2_pin = 7 # BOARD pin 7, K2 버튼

SW_UP_pin = 17 # Camera Lifter 상단 Limit Switch
SW_DOWN_pin = 18 # Camera Lifter 하단 limit Switch

GPIO.setmode(GPIO.BCM) # 보드 핀 번호 방식

GPIO.setup(BEEP_pin, GPIO.OUT, initial=GPIO.LOW) # 초기설정 = 끄기

GPIO.setup(GREEN_pin, GPIO.OUT) # 초록 LED Pin set as output
GPIO.setup(BLUE_pin, GPIO.OUT) # 파랑 LED Pin set as output

GPIO.setup(KEY1_pin, GPIO.OUT) # KEY1 버튼 Pin set as output
GPIO.setup(KEY2_pin, GPIO.OUT) # KEY2 버튼 Pin set as output

/usr/local/lib/python3.6/dist-packages/Jetson/GPIO/gpio.py:352: RuntimeWarning: This channel is already in use, continuing anyway. Use GPIO.setwarnings(False) to disable warnings
  RuntimeWarning)


In [8]:
### 지정 시간 동안 부저음 울리기
def Beep(beepTime):
    GPIO.output(BEEP_pin, GPIO.HIGH) # 부저 켜기
    time.sleep(beepTime) # 소리 내는 시간
    GPIO.output(BEEP_pin, GPIO.LOW) # 부저 끄기
    
### 보드상의 초록 LED on/off
def OnOffGreen(bOn):
    if bOn:
        GPIO.output(GREEN_pin, GPIO.LOW) # GREEN LED 켜기
    else:
        GPIO.output(GREEN_pin, GPIO.HIGH) # GREEN LED 켜기
                    
### 보드상의 파랑 LED on/off
def OnOffBlue(bOn):
    if bOn:
        GPIO.output(BLUE_pin, GPIO.LOW) # BLUE LED 켜기
    else:
        GPIO.output(BLUE_pin, GPIO.HIGH) # BLUE LED 켜기
                    


In [9]:
OnOffGreen(1)
OnOffBlue(1)
Beep(1)

In [12]:
# 기둥 위아래로 moving

from jetbot import Robot
robot = Robot()

OnOffGreen(1)
OnOffBlue(1)

value = GPIO.input(KEY1_pin) # GPIO.LOW or GPIO.HIGH
value = GPIO.input(KEY2_pin) # GPIO.LOW or GPIO.HIGH

GPIO.setwarnings(False)
GPIO.setup(SW_UP_pin, GPIO.IN)
GPIO.setup(SW_DOWN_pin, GPIO.IN)

# if GPIO.input(SW_UP_pin):
#     robot.down(1)
# else:
#     robot.vertical_motors_stop()
    
if GPIO.input(SW_DOWN_pin):
    robot.up(1)
else:
    robot.vertical_motors_stop()
    
# GPIO.cleanup()

### 모터/서보 성능 및 위치 기본 조정값 선언

In [13]:
L_F_MPR = 0.93 # 좌측 전진 모터 성능 비율 (Left_forward_Motor_Performance_Ratio)
R_F_MPR = 1.00 # 우측 전진 모터 성능 비율 (Right_forward_Motor_Performance_Ratio)

L_B_MPR = 0.98 # 좌측 후진 모터 성능 비율 (Left_Backward_Motor_Performance_Ratio)
R_B_MPR = 1.00 # 우측 후진 모터 성능 비율 (Right_Backward_Motor_Performance_Ratio)

PAN_ADJ = 0 # 카메라 팬 서보 위치 조정값(Camera_Pan_Servo_Adjust)
TILT_ADJ = 0 # 카메라 틸트 서보 위치 조정값(Camera_Tilt_Servo_Adjust)

##### 모터/서보 성능 위치 및 조정정보 설정파일 읽기
생략

### PanTilt 서보 인스턴스 생성

In [14]:
import os

os.system("sudo chmod 777 /dev/ttyTHS1")

from servoserial import ServoSerial
pantilt = ServoSerial() # 팬틸트 서보 컨트롤

serial Open!


##### PAN, TILT 값 정의

In [16]:
MID_TILT = 3600
# TILT 범위 3000~4095
MID_PAN = 4095 // 2 + 100

DEF_TILT = 4000

### 초기 카메라 팬/틸트 위치 설정


In [20]:
pantilt.Servo_serial_double_control(1, MID_PAN, 2, MID_TILT-600) # 장애물회피 주행용 팬틸트 위치
time.sleep(1)

pantilt.Servo_serial_double_control(1, MID_PAN, 2, MID_TILT) # 중앙정면 팬틸트 위치
time.sleep(1)

pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT) # 중앙상향 얼굴인식 팬틸트 위치
time.sleep(1)

47
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0b\x86\x00\n/'
162
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0e\x10\x00\n\xa2'
17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'


### Robot 인스턴스는 위에서 생성했음

##### Camera Lifter 테스트
젯봇 전면의 카메라 높낮이를 조정하는 리프터 상하 이동 제어

In [22]:
# robot.up(1.0)
# time.sleep(3)
# robot.vertical_motors_stop()

# robot.down(1.0)
# time.sleep(3)
# robot.vertical_motors_stop()

##### 기본 주행 속력과 주행 및 정지 함수 정의

In [26]:
Speed = 0.7 ## 최대 속력의 70% 빠르기를 기본 속력으로 설정

### 좌우 모터의 속력을 따로 지정하여 주행하기

def Move2(speedLeft, speedRight):
    if speedLeft > 0:
        robot.left_motor.value = speedLeft * L_F_MPR # 전진 시 왼쪽 모터의 전진 성능 비율 적용
    else:
        robot.left_motor.value = speedLeft * L_B_MPR # 후진 시 왼쪽 모터의 후진 성능 비율 적용
    
    if speedRight > 0:
        robot.right_motor.value = speedRight * R_F_MPR # 전진 시 오른쪽 모터의 전진 성능 비율 적용
    else:
        robot.right_motor.value = speedRight * R_B_MPR # 후진 시 오른쪽 모터의 후진 성능 비율 적용
    
### 좌우 모터의 속력을 동일하게 지정하여 주행하기

def Move(speed):
    Move2(speed, speed)
    
### 이동 정지
def Stop():
    robot.left_motor.value = 0
    robot.right_motor.value = 0
    


##### MODE_PERSON 동작: 사람이 감지되면 반가워서 인사하는 동작
사람이 마지막에 감지된 시각으로부터 특정 시간보다 더 지나서 다시 사람이 감지되면 동작됨

In [38]:
ActTimePeriodPerson = 30 # 30초 이상 사람이 감지되지 않다가 다시 감지되면 동작
LastTimeDetectedPerson = 0 # 가장 최근에 사람이 감지된 시각

def ActModePerson(repeat=3):
    speed = 3
    delay = 0.02
    
    RGB.Set_BreathSColor_RGB(6) # Color: 0(red), 1(green), 2(blue), 3(yellow), 4(magenta), 5(cyan), 6(white)
    RGB.Set_BreathSSpeed_RGB(3) # speed: 1(slow), 2(normal), 3(fast)
    RGB.Set_BreathSLight_RGB()
    
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT) # 중앙상향 얼굴인식 팬틸트 위치
    Beep(0.05)
    
    for i in range(0, repeat):
        pantilt.Servo_serial_control(1, MID_PAN+200) # Left
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+400)
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+200) 
        Stop()
        time.sleep(delay)
        
        pantilt.Servo_serial_control(1, MID_PAN-200) # Right
        Move2(speed, -speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-400)
        Move2(speed, -speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-200) 
        Stop()
        time.sleep(delay)
        
        pantilt.Servo_serial_control(1, MID_PAN)
        Stop()
        time.sleep(delay)
        
    for i in range(0, 2):
        pantilt.Servo_serial_control(2, DEF_TILT+100)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT+200)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT+100)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-200)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT)
        time.sleep(delay)
    
    Beep(0.05)
    time.sleep(0.1)
    Beep(0.05)
    
    RGB.OFF_ALL_RGB() # 전체 LED 끄기
    RGB.Set_All_RGB(0x00, 0x00, 0x10) # 기본 색상 어두운 파랑으로

In [40]:
ActModePerson()

17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
196
b'\xff\xff\x01\x07\x03*\t\xf3\x00\n\xc4'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
231
b'\xff\xff\x01\x07\x03*\x06\xd3\x00\n\xe7'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
85
b'\xff\xff\x01\x07\x03*\x08c\x00\nU'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
196
b'\xff\xff\x01\x07\x03*\t\xf3\x00\n\xc4'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
231
b'\xff\xff\x01\x07\x03*\x06\xd3\x00\n\xe7'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
85
b'\xff\xff\x01\x07\x03*\x08c\x00\nU'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
196
b'\xff\xff\x01\x07\x03*\t\xf3\x00\n\xc4'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
231
b'\xff\xff\x01\x07\x03*\x06\xd3\x00\n\xe7'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
85
b'\xff\xff\x01\x07\x03*\x08c\x00\nU'
177
b'\

##### ActModeFruit: 과일 먹는 동작

In [55]:
ActTimePeriodFruit = 30 # 30초 이상 컵이 감지되지 않다가 다시 감지되면 동작
LastTimeDetectedFruit = 0 # 가장 최근에 컵이 감지된 시각

def ActModeFruit(repeat = 2):
    speed = 0.4
    delay = 0.05
    
    RGB.Set_BreathSColor_RGB(1) # Color : 0(red), 1(green), 2(blue), 3(yellow), 4(magenta), 5(cyan), 6(white)
    RGB.Set_BreathSSpeed_RGB(3) # speed : 1(slow), 2(normal), 3(fast)
    RGB.Set_BreathSLight_RGB()
    
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT)
    
    for i in range(0, repeat):
        pantilt.Servo_serial_control(2, DEF_TILT-100) # down
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-200) # down
        Move2(speed, -speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-300) # down
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-400) # down
        Move2(speed, -speed)
        time.sleep(delay)
        Stop()
        
        # 고개 숙이는 동작
        pantilt.Servo_serial_double_control(1, MID_PAN-50, 2, DEF_TILT-500) 
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-300) 
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN+50, 2, DEF_TILT-500) 
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-300) 
        time.sleep(delay)
        
        pantilt.Servo_serial_double_control(1, MID_PAN-50, 2, DEF_TILT-500)
        Move2(speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-500)
        Move2(-speed, -speed)
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN+50, 2, DEF_TILT-500)
        Move2(speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-500)
        Move2(-speed, -speed)
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN-50, 2, DEF_TILT-500)
        Stop()
        time.sleep(delay)
        
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-300) 
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN+50, 2, DEF_TILT-500) 
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-300) 
        time.sleep(delay)
        
        pantilt.Servo_serial_control(2, DEF_TILT-400)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-300)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-200)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-100)
        time.sleep(delay)
        
        pantilt.Servo_serial_control(2, DEF_TILT)
        time.sleep(delay)
        
        # 고개 들어올림
        pantilt.Servo_serial_control(1, MID_PAN+100) # left
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+200)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+100)
        time.sleep(delay)
        
        pantilt.Servo_serial_control(1, MID_PAN-100) # right
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-200) 
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-100) 
        time.sleep(delay)
        
        pantilt.Servo_serial_control(1, MID_PAN)
        
        RGB.OFF_ALL_RGB() # 전체 LED 끄기
        RGB.Set_All_RGB(0x00, 0x00, 0x10) # 기본 색상으로 켜기(어두운 파랑)

In [56]:
ActModeFruit()

17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
116
b'\xff\xff\x02\x07\x03*\x0f<\x00\nt'
217
b'\xff\xff\x02\x07\x03*\x0e\xd8\x00\n\xd9'
61
b'\xff\xff\x02\x07\x03*\x0et\x00\n='
161
b'\xff\xff\x02\x07\x03*\x0e\x10\x00\n\xa1'
57
b'\xff\xff\xfe\x0e\x83*\x04\x01\x081\x00\n\x02\r\xac\x00\n9'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0et\x00\n>'
213
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x95\x00\n\x02\r\xac\x00\n\xd5'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0et\x00\n>'
57
b'\xff\xff\xfe\x0e\x83*\x04\x01\x081\x00\n\x02\r\xac\x00\n9'
7
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\r\xac\x00\n\x07'
213
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x95\x00\n\x02\r\xac\x00\n\xd5'
7
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\r\xac\x00\n\x07'
57
b'\xff\xff\xfe\x0e\x83*\x04\x01\x081\x00\n\x02\r\xac\x00\n9'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0et\x00\n>'
213
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x95\x00\n\x02\r\xac\x00\n\xd5'
62
b'\xff\xff\xfe\x0e\x83*\x04